# <font face="仿宋">课程说明：

&emsp;&emsp;<font face="仿宋">小伙伴好呀\~欢迎来到《机器学习实战训练营》试学体验课！我是课程主讲老师，九天。本期公开课主题为《模型融合与深度森林》，将重点探讨时下最为流行的模型融合及其优化方法，以及机器学习最新理论成果——深度森林算法的基本原理与实践方法。         

- 课件领取

&emsp;&emsp;<font face="仿宋">公开课所有课件、代码和数据，均可在直播期间免费获取。扫描二维码添加官方客服小可爱，或者手动添加微信号littlebird_0228，并回复“ML618”即可获取哦～

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h2yiv3wpxjj20ox0p5dhg.jpg" alt="二维码" style="zoom:33%;" />

- 节选自《机器学习实战训练营》付费正课

&emsp;&emsp;<font face="仿宋">公开课所有内容都节选自《机器学习实战训练营》付费正课。付费课程为130+小时完整体系大课，包括经典机器学习算法、集成算法、特征工程、模型融合、Kaggle竞赛案例与企业实战案例六大板块内容，想要获得更加完整、深入的学习体验，也欢迎大家报名《机器学习实战训练营》付费正课哦～

- 限时折扣

&emsp;&emsp;<font face="仿宋">《机器学习实战训练营》付费正课目前正在618年中大促中，限时半价！私聊小可爱回复“优惠”，还可领取直播限定折上折优惠哦～课纲&介绍&试学，详情请戳👉https://appze9inzwc2314.pc.xiaoe-tech.com

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h2yjdm3r92j21qe0f8gsr.jpg" alt="二维码" style="zoom:33%;" />

---

# <center> 《模型融合与深度森林专题公开课》

# <center>Day 3.深度森林原理与实践

- 导入相关库

In [125]:
# 基础数据科学运算库
import numpy as np
import pandas as pd
from numpy.random import RandomState

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 其他辅助模块
import inspect, re
from tqdm import tqdm
import gc
import time
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

# hyperopt优化器
from hyperopt import hp, fmin, tpe

&emsp;&emsp;接下来我们正式进入到深度森林相关内容。在本节内容中，我们将重点探讨机器学习最新研究成果——深度森林算法的基本原理及其实践技巧。当然，正如公开课开篇所言，我们学习深度森林，不只是掌握的算法增加一项，更关键的是，深度森林的本质其实是一种基于随机森林的模型融合策略，通过学习深度森林，我们从中窥见一些模型融合的核心思想和精彩策略，对于提升我们的模型融合技巧也有着至关重要的作用。

## 一、深度森林算法原理

- 方法提出

&emsp;&emsp;不可否认，深度森林的提出很大程度是受深度学习算法启发。现如今，深度学习算法在诸多领域都展示出了傲人的实力，周志华教授作为国内集成学习领域的先驱，则在借鉴了深度学习算法结构的基础上，提出了深度森林算法。我们可以说深度森林是深度学习算法的一种变种，但按照周教授的说法，更准确的来说，深度森林应该是集成学习的一个重大突破。深度森林论文已共享在公开课的网盘链接中，接下来我们对照论文进行介绍。

&emsp;&emsp;根据周志华教授的观点，深度神经网络的成功主要归结为三点，分别是逐层处理（layer-by-layer processing）、模型内特征变化（in-model feature transformation）以及模型复杂度（sufficient model complexity）。而在此基础之上，如果将神经网络中可微模块替换成不可微模块，例如随机森林，则能够在确保模型有效性和稳定性的基础上，减少模型对于反向传播的依赖。并且经过实验证明，采用类似神经网络的结果进行随机森林的集成，能够获得一个超参数效果稳定、算法复杂性可控的全新的模型——这个模型就是深度森林。

&emsp;&emsp;深度森林又名多粒度扫描的级联森林，这也是深度森林中最核心的两个概念，其一是级联森林，其二则是多粒度扫描。

### 1.基本结构：级联森林（cascade forest）

&emsp;&emsp;所谓的级联森林，指的构建一种类似神经网络的处理结构，如下所示，其中级联的每一级接收其前一级处理的特征信息，并将其处理结果输出到下一级。

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h30py4j87mj219o0ka0w3.jpg" alt="二维码" style="zoom:33%;" />

而和神经网络不同的是，这里构成网络状结构的节点不再是激活函数，而是随机森林。其中每一层的深度森林包括两种模型，完全随机森林（completely-random tree forests）（蓝色）和随机森林（random forests）（黑色），在实际使用过程中，每一层两类模型的种类是超参数，可以人工调节。上图为了见简化其间，展示了两个随机森林与两个极端随机树集成的情况。并且在默认情况下，每个模型都包含500个弱分类器，即完全随机森林包含500个完全随机决策树，而随机森林模型则包含500个决策树模型。并且在默认情况下，每个模型只接收$\sqrt{d}$个特征，其中d是总特征数。

&emsp;&emsp;上述结构实际上就是一个集成算法的集成（ensemble of ensembles），而$\sqrt{d}$个特征的分配，实际上也是继承自随机森林的参数设置经验。

### 2.单模型训练与强化特征矩阵

&emsp;&emsp;然后，非常关键的环节是，在实际训练过程，每个随机森林模型（也包括完全随机森林）的输出并不是模型的预测概率，而是叶节点中不同样本的数量占比，并且是按照每一条数据来进行的计算。例如当我们带入全部数据集进行一个随机森林进行计算时，假设该随机森林只有3颗树（实际默认情况是500颗树，上述情况只是简化情况），那么在每一棵树中，某条条数据x（也就是某个样本）一定被分在了树的某个叶节点上，例如下图所示的第一棵树的最左边叶节点、第二棵树的中间叶节点、第三棵树的右边叶节点，如下所示：

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h30tcx31hyj213i0ewgnq.jpg" alt="二维码" style="zoom:33%;" />

此时样本总共有三个不同类别，分别是绿色圆圈、红色的方框和黄色的三角，第一棵树上叶节点上三个不同类别样本占比分别为0.2：0.5：0.3，第二棵树上叶节点中三类占比为0.3：0.4：0.3，第三棵上叶节点中三类占比为1：0：0，然后计算三个类别占比的均值，即：

In [34]:
(np.array([0.2, 0.5, 0.3]) + np.array([0.3, 0.4, 0.3]) + np.array([1, 0, 0])) / 3

array([0.5, 0.3, 0.2])

由此x样本就可以获得额外的3个特征（也被称作增强特征），即0.5、0.3、0.2。需要注意的是，每个样本都可以按照这个规则进行计算，假设当前数据集有m条数据，那么把这m条数据各自的增强特征按照样本顺序进行拼接，则可以拼接成规模为m行3列的增强特征集。即按照上述规则，一个随机森林即可创造一个包含3个特征的数据集，而如果进一步增加模型数量，即带入两个随机森林和两个完全随机森林进行计算，则可以创造包含3\*4=12个特征的增强特征数据集。

&emsp;&emsp;这里需要简单讨论，为何要求每个评估器输出叶节点上不同类别样本占比而不是模型预测结果，这里其实有两层考虑：首先，对于一个随机森林集成算法的集成，其本质仍然还是借助决策树的分类原理进行预测，即希望通过某些规则的叠加最终提升叶节点的不同样本类别的纯度；其次由于当前算法是借鉴了神经网络算法结构，而神经网络算法的核心优势就在于不同层其实能进行“有侧重”的学习，而其实深度森林也希望最终模型的不同层能呈现出“有侧重”的学习情况。当然，这也是后续模型为何要多粒度扫描的核心原因。

&emsp;&emsp;当然，与其说模型输出的是样本预测结果，不如说此时每个单独的随机森林输出的是某种形式的模型分类效力指标。无论如何，每个模型的输出结果将成为一个增强特征（augmented features），当该层有4个模型时候，下一层将接收到12个增强特征。不难看出，这种增强特征数量较少，而在深度森林论文中，作者给出了这种增强特征“少但有效”的证明，我们只需要知道其确实有效即可。此外，为了减少过拟合风险，一般来说深度森林的每个随机森林在进行计算时也是需要经过交叉验证的，即得到的样本占比的均值，实际上是经过交叉验证后得到的结果。

### 3.滑动窗口与多粒度扫描（Multi-Grained Scanning）

&emsp;&emsp;在了解了每个模型的训练过程和输出结果之后，接下来进一步看不同层的训练过程。这里最重要的概念就是多粒度扫描（Multi-Grained Scanning）。所谓的多粒度扫描，实际上指的是每一层接收到的特征和样本各不相同，从数据集角度来看，这个过程就类似于随着模型训练，逐渐扫描二维数据集。很明显，这个过程也是受到卷机神经网络和循环神经网络的启发——通过滑动窗口来学习完整数据集的信息。

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h30ueuv7caj219i0rqgqj.jpg" alt="二维码" style="zoom:33%;" />

&emsp;&emsp;先举个简单的例子了解什么是滑动窗口，假设现在某数据集有5个特征，我们以两个特征为窗口大小，滑动窗口的过程总共能创造4个包含两个特征的数据集：

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h30vpc43z8j21460beab3.jpg" alt="二维码" style="zoom:33%;" />

类似的，如果现在数据集有100个特征，并且以10为窗口大小，滑动过程中就能创建301个包含100个特征的数据集。这其实就相当于是一种划分数据集的方法。

&emsp;&emsp;接下来，针对这301个包含100个特征的数据集，我们可以按照之前所说的单模型训练过程，逐个带入这些数据集进行训练。注意，虽然是训练，但实际这也是一个数据增强的过程，即每个数据集训练完毕之后都将创造一个m\*3的强化特征矩阵（假设总共是m条样本），而总共有301个数据集，因此一个模型就将产生一个包含301\*3=903个特征的强化特征矩阵。而如果是带入两个模型，则将创造1806个特征的强化特征矩阵，即创造一个m\*1806的数据集。注意该数据集也被称为原始强化数据集，需要带入到后续模型中进行训练。

### 4.深度森林的模型训练和预测过程

&emsp;&emsp;在了解了强化特征矩阵的创建过程后，接下来我们进一步来讨论如何进行级联训练。整个训练过程可以由下图进行表示，我们逐部分进行介绍。

<center><img src="https://tva1.sinaimg.cn/large/e6c9d24ely1h30vyaxuamj21wm0rw101.jpg" alt="二维码" style="zoom:33%;" />

- 通过多粒度扫描创建原始增强特征矩阵

&emsp;&emsp;首先是通过多粒度扫描创建原始增强特征矩阵。需要注意的是，之前我们介绍过，在原始数据集有400个特征，并且窗口大小在100的情况下，通过一个随机森林和一个完全随机森林两个模型，就能创造一个m\*1806的增强特征矩阵。而这个滑动窗口的大小是可以调整的，例如如果将窗口大小设置为200，则能创造201个包含200个特征的的数据集，按照每个数据集带入一个模型能创建3个增强特征的情况来进行计算，201个数据集\*3\*2个模型=能创建1206个特征的增强数据集。类似的如果窗口为300，则总共能创建101个数据集，带入两个模型则最终能创建一个包含606个特征的增强数据集。

&emsp;&emsp;总的来看，在三个不同窗口大小设置的情况下，最终我们能获得A、B、C三个增强特征数据集。接下来我们就用这三个数据集来进行级联模型的训练。当然需要注意的是，这里选择几个窗口、每个窗口选择多少特征，都是可以作为超参数进行人工调整的。

- 级联模型训练

&emsp;&emsp;在准备了基础数据集之后，接下来开始进行级联模型训练。如上图所示，每一层我们都打算训练一组2个随机森林+2个完全随机森林的模型组，这里我们首先带入数据集A完成这第一组组模型的训练，训练完成后该组模型同样也将创建一个m\*12的增强特征数据集。我们将该数据集与A数据集拼接，就得到了一个m\*1818增强数据集，然后以该数据集训练第二层模型组，注意，每一层的模型组都是相同的，第二层模型组仍然是2个随机森林+2个完全随机森林，并且第二层模型组的训练和第一层模型组训练过程完全相同，在训练结束后将创造一个m\*12增强特征矩阵，我们将其和数据集B进行拼接，最终构成一个m\*1218数据集，并带入到第三层进行计算。第三层计算过程也类似，结束后得到一个C矩阵拼接后的增强特征矩阵（$Level 1_C$），并且得到一组预测结果。此时我们可以根据实际情况是否构建第二层的级联模型，如果继续构建下一层级联模型，则以$Level 1_C$作为初始矩阵进行带入训练，再次进行训练，如此往复。整体来看，这个过程其实就是级联的级联。

- 收敛条件

&emsp;&emsp;那么何时训练停止呢？一方面我们可以根据实际硬件情况决定迭代停止条件，此外也可以比较两个相邻的级联结果差异，如果发现增加一个级联并不能大幅提升模型效果，则可以考虑停止增加级联。

- 预测过程

&emsp;&emsp;而当我们完成了模型训练之后，接下来就可以进行预测了。实际上深度森林的预测过程会非常高效，当带入某条测试数据时，我们只需要根据最终预测预测结果，即先寻找该样本的最高评分（即某个随机森林模型的叶节点的最高平均占比），然后根据该叶节点的类别判定当前样本所属类别即可。

## 二、深度森林算法实践

### 1.深度森林安装与使用

- 深度森林算法库安装

&emsp;&emsp;首先我们可以使用pip快速安装深度森林算法库：

<center> pip install deep-forest -i https://pypi.tuna.tsinghua.edu.cn/simple

然后即可按照如下方式进行导入。

In [39]:
from deepforest import CascadeForestClassifier

In [41]:
CascadeForestClassifier?

Init signature:
CascadeForestClassifier(
    n_bins=255,
    bin_subsample=200000,
    bin_type='percentile',
    max_layers=20,
    criterion='gini',
    n_estimators=2,
    n_trees=100,
    max_depth=None,
    min_samples_leaf=1,
    use_predictor=False,
    predictor='forest',
    predictor_kwargs={},
    backend='custom',
    n_tolerant_rounds=2,
    delta=1e-05,
    partial_mode=False,
    n_jobs=None,
    random_state=None,
    verbose=1,
)
Docstring:     
Implementation of the deep forest for classification.


Parameters
----------
n_bins : :obj:`int`, default=255
    The number of bins used for non-missing values. In addition to the
    ``n_bins`` bins, one more bin is reserved for missing values. Its
    value must be no smaller than 2 and no greater than 255.
bin_subsample : :obj:`int`, default=200,000
    The number of samples used to construct feature discrete bins. If
    the size of training set is smaller than ``bin_subsample``, then all
    training samples will be used

当然，如果是回归问题，则可以导入回归模型：

In [48]:
from deepforest import CascadeForestRegressor

- 深度森林算法初步实践

&emsp;&emsp;深度森林整体风格和sklearn非常一致，我们可以尝试导入数据集进行建模预测：

In [88]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, random_state=1)
model = CascadeForestClassifier(random_state=8)
model.fit(X_train1, y_train1)
y_pred = model.predict(X_test1)
acc = accuracy_score(y_test1, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2022-06-08 09:28:13.719] Start to fit the model:
[2022-06-08 09:28:13.719] Fitting cascade layer = 0 
[2022-06-08 09:28:14.169] layer = 0  | Val Acc = 97.921 % | Elapsed = 0.450 s
[2022-06-08 09:28:14.173] Fitting cascade layer = 1 
[2022-06-08 09:28:14.629] layer = 1  | Val Acc = 98.144 % | Elapsed = 0.456 s
[2022-06-08 09:28:14.631] Fitting cascade layer = 2 
[2022-06-08 09:28:15.029] layer = 2  | Val Acc = 97.699 % | Elapsed = 0.397 s
[2022-06-08 09:28:15.029] Early stopping counter: 1 out of 2
[2022-06-08 09:28:15.031] Fitting cascade layer = 3 
[2022-06-08 09:28:15.418] layer = 3  | Val Acc = 96.956 % | Elapsed = 0.387 s
[2022-06-08 09:28:15.418] Early stopping counter: 2 out of 2
[2022-06-08 09:28:15.418] Handling early stopping
[2022-06-08 09:28:15.418] The optimal number of layers: 2
[2022-06-08 09:28:15.419] Start to evalute the model:
[2022-06-08 09:28:15.419] Evaluating cascade layer = 0 
[2022-06-08 09:28:15.440] Evaluating cascade layer = 1 

Testing Accuracy: 98.444 %


尽管深度森林原理复杂，但实际的使用过程秉承了sklearn的一贯风格——代码简单，并且拥有一整套效果较好的初始化参数。接下来简单对比决策树和随机森林模型效果：

In [94]:
# 决策树模型训练过程
model = DecisionTreeClassifier(random_state=8)
model.fit(X_train1, y_train1)
y_pred = model.predict(X_test1)
acc = accuracy_score(y_test1, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 84.889 %


In [93]:
# 随机森林模型训练过程
model = RandomForestClassifier(random_state=8)
model.fit(X_train1, y_train1)
y_pred = model.predict(X_test1)
acc = accuracy_score(y_test1, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 98.222 %


In [97]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [110]:
# XGBoost模型预测结果
model = XGBClassifier(random_state=8)
model.fit(X_train1, y_train1)
y_pred = model.predict(X_test1)
acc = accuracy_score(y_test1, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 97.111 %


In [116]:
# LGBM模型预测结果
model = LGBMClassifier(random_state=8)
model.fit(X_train1, y_train1, categorical_feature = 'auto',eval_set=(X_test1,y_test1),feature_name='auto', verbose=0)
y_pred = model.predict(X_test1)
acc = accuracy_score(y_test1, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 96.444 %


能够看出深度森林实际效果。

|Models|test_score|
|:--:|:--:|
|CascadeForest|98.444 %|
|DecisionTree|84.889 %|
|RandomForest|98.222 %|
|XGBoost|97.111 %|
|LGBM|96.444 %|

- 深度森林算法抗过拟合特性

&emsp;&emsp;此外，深度森林其实拥有天然的抗过拟合特性（自带交叉验证过程）。因此在不进行任何参数调整情况下，在当前案例数据集上也能得到一个较好的结果：

In [87]:
model = CascadeForestClassifier(random_state=8)
model.fit(X_train_OE.values, y_train.values)
y_pred = model.predict(X_test_OE)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2022-06-08 09:26:52.932] Start to fit the model:
[2022-06-08 09:26:52.932] Fitting cascade layer = 0 
[2022-06-08 09:26:53.785] layer = 0  | Val Acc = 79.137 % | Elapsed = 0.853 s
[2022-06-08 09:26:53.790] Fitting cascade layer = 1 
[2022-06-08 09:26:54.929] layer = 1  | Val Acc = 80.178 % | Elapsed = 1.139 s
[2022-06-08 09:26:54.934] Fitting cascade layer = 2 
[2022-06-08 09:26:56.085] layer = 2  | Val Acc = 80.121 % | Elapsed = 1.151 s
[2022-06-08 09:26:56.085] Early stopping counter: 1 out of 2
[2022-06-08 09:26:56.090] Fitting cascade layer = 3 
[2022-06-08 09:26:57.245] layer = 3  | Val Acc = 80.329 % | Elapsed = 1.155 s
[2022-06-08 09:26:57.250] Fitting cascade layer = 4 
[2022-06-08 09:26:58.403] layer = 4  | Val Acc = 79.799 % | Elapsed = 1.153 s
[2022-06-08 09:26:58.403] Early stopping counter: 1 out of 2
[2022-06-08 09:26:58.408] Fitting cascade layer = 5 
[2022-06-08 09:26:59.557] layer = 5  | Val Acc = 80.140 % | Elapsed = 1.149 s
[2022-06-08 09:26:59.557] Early stopping c

能够发现，基本能够获得一个和随机森林调参后类似的结果。

In [115]:
# XGBoost模型预测结果
model = XGBClassifier(random_state=8)
model.fit(X_train_OE, y_train)
y_pred = model.predict(X_test_OE)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 77.740 %


In [119]:
# LGBM模型预测结果
model = LGBMClassifier(random_state=8)
model.fit(X_train_OE, y_train, categorical_feature = 'auto',eval_set=(X_test_OE,y_test),feature_name='auto', verbose=0)
y_pred = model.predict(X_test_OE)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 78.705 %


In [134]:
# CatBoost模型预测结果
categorical_features_indices = np.where(X_train_OE.dtypes != np.float)[0]

model = CatBoostClassifier(verbose=False,random_state=8)
model.fit(X_train_OE, y_train,cat_features=categorical_features_indices,eval_set=(X_test_OE, y_test))
y_pred = model.predict(X_test_OE)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))


Testing Accuracy: 79.273 %


|Models|test_score|
|:--:|:--:|
|CascadeForest|79.046 %|
|XGBoost|77.740 %|
|LGBM|78.705%|
|CatBoost|79.273 %|